In [54]:
import pandas as pd
import numpy as np
import country_converter as coco
from pandas_profiling import ProfileReport 
import re

In [2]:
# Define file location
FILE_LOCATION = "Dataset.csv"
# USE_COLS = ['DATE', 'REGION', 'CONTAMINES', 'DECES', 'GUERIS', 'CONTAMINES_FEMME', 'CONTAMINES_HOMME', 'CONTAMINES_GENRE_NON_SPECIFIE'] 

# Load the CSV as a pandas dataframe, but only selected columns
df = pd.read_csv(FILE_LOCATION, delimiter=",", low_memory=False)
# , usecols=USE_COLS)
df = df.drop("Country", 1)
df = df.drop("Source", 1)
df.head()

,Country code,Case & death data by sex?,Cases date,Cases where sex-disaggregated data is available,Cases (% male),Cases (% female),Deaths date,Deaths where sex-disaggregated data is available,Deaths (% male),Deaths (% female),Deaths in confirmed cases date,Proportion of deaths in confirmed cases (male),Proportion of deaths in confirmed cases (female),Proportion of deaths in confirmed cases (Male:female ratio)
0,AF,Yes,2021/02/02,52189.0,67.57%,32.43%,2021/02/02,1935.0,72.66%,27.34%,2021/02/02,3.99%,3.13%,1.28
1,AL,Yes,2021/02/01,78992.0,48%,52%,2021/02/01,1393.0,67%,33%,2021/02/01,2.46%,1.12%,2.20
2,DZ,Partial,2020/11/12,55630.0,53.8%,46.2%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AD,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AO,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df['Geography'] = ""

for i, row in df.iterrows():
    iso2 = row['Country code']
    iso3 = coco.convert(names=iso2, to='ISO3')
    df.at[i,'Geography'] = iso3
    
df.head()

nan not found in ISO3
nan not found in ISO3
nan not found in ISO3
SAO not found in ISO3
nan not found in ISO3
SEY not found in ISO3
nan not found in ISO3


,Country code,Case & death data by sex?,Cases date,Cases where sex-disaggregated data is available,Cases (% male),Cases (% female),Deaths date,Deaths where sex-disaggregated data is available,Deaths (% male),Deaths (% female),Deaths in confirmed cases date,Proportion of deaths in confirmed cases (male),Proportion of deaths in confirmed cases (female),Proportion of deaths in confirmed cases (Male:female ratio),Geography
0,AF,Yes,2021/02/02,52189.0,67.57%,32.43%,2021/02/02,1935.0,72.66%,27.34%,2021/02/02,3.99%,3.13%,1.28,AFG
1,AL,Yes,2021/02/01,78992.0,48%,52%,2021/02/01,1393.0,67%,33%,2021/02/01,2.46%,1.12%,2.20,ALB
2,DZ,Partial,2020/11/12,55630.0,53.8%,46.2%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DZA
3,AD,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AND
4,AO,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGO


In [11]:
cases = df[["Geography", "Cases (% male)", "Cases (% female)"]]
cases = cases.melt(id_vars=["Geography"], 
        var_name="Indicator", 
        value_name="Count")

cases['Count'].replace('%','',regex=True,inplace=True)

cases = cases.replace(to_replace ="Cases (% male)", 
                 value ="Male") 
cases = cases.replace(to_replace ="Cases (% female)", 
                 value ="Female") 

cases = cases.astype(object).replace(np.nan, 'Null')
cases.to_csv(r'./output/Gender_Cases.csv', index = False, sep=',')

In [12]:
deaths = df[["Geography", "Deaths (% male)", "Deaths (% female)"]]
deaths = deaths.melt(id_vars=["Geography"], 
        var_name="Indicator", 
        value_name="Count")

deaths['Count'].replace('%','',regex=True,inplace=True)

deaths = deaths.replace(to_replace ="Deaths (% male)", 
                 value ="Male") 
deaths = deaths.replace(to_replace ="Deaths (% female)", 
                 value ="Female") 

deaths = deaths.astype(object).replace(np.nan, 'Null')
deaths.to_csv(r'./output/Gender_Deaths.csv', index = False, sep=',')

In [14]:
ratio = df[["Geography", "Proportion of deaths in confirmed cases (Male:female ratio)"]]
ratio = ratio.melt(id_vars=["Geography"], 
        var_name="Indicator", 
        value_name="Count")

ratio = ratio.replace(to_replace ="Proportion of deaths in confirmed cases (Male:female ratio)", 
                 value ="Male:female ratio")

ratio = ratio.astype(object).replace(np.nan, 'Null')
ratio.to_csv(r'./output/Gender_Ratio.csv', index = False, sep=',')